<a href="https://colab.research.google.com/github/EP111111/Doc_analysis/blob/main/DA_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth as google_auth
google_auth.authenticate_user()


In [2]:
!pip install shapely==1.8.5


In [ ]:

!pip install google-cloud-aiplatform --upgrade
!pip install python-docx
!pip install langchain
!pip install docx2txt
!pip install chromadb

In [51]:
import vertexai
from vertexai.language_models import TextGenerationModel
from google.colab import drive
import docx
import numpy as np
import pandas as pd
from functions import *

In [42]:
import time
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import Chroma
import pandas as pd
import json

In [44]:
PROJECT_ID = "ics-analysis-dev"
vertexai.init(project=PROJECT_ID, location="us-central1")

model = TextGenerationModel.from_pretrained("text-bison@001")


##Document Splitting
Document Splitting is to split documents into smaller chunks.

In [8]:
from langchain.document_loaders import Docx2txtLoader
text=Docx2txtLoader("Project Alpha.docx").load()

In [86]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=2000,
    chunk_overlap=150,
    length_function=len
)

docs = text_splitter.split_documents(text)


##Vector Stores and Embeddings
 put these chunks into an index so that we are able to retrieve them easily when we want to answer questions on this document.

In [87]:
embeddings = VertexAIEmbeddings()
# Create the vector store
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="/content"
)

print(vectordb._collection.count())

142


In [88]:
from langchain.chat_models import ChatVertexAI
llm = ChatVertexAI()

In [89]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [90]:
question = """extract all questions in this document  :
"""
result = qa_chain({"query": question})
print(result["result"])

import re

questions = re.split(r'\d+\.', result["result"])
questions = [q.strip() for q in questions if q.strip()]

# for i, question in enumerate(questions, start=1):
#     print(f"{i}. {question}")

 1. What is the purpose of this RFP? 
2. What is the background of the service? 
3. What are the general information about Project Alpha? 
4. What are the terms and conditions of the project? 
5. What are the high-level project requirements? 
6. What vendor information is required? 
7. What is the vendor proposal? 
8. What is the method of evaluations and award? 
9. What are the attachments? 
10. What is the timesheet allocation worksheet? 
11. What is the activity based


In [92]:
print(len(docs))


22


## find the context to the questions in each different part

EXTRACT context related to the question

In [93]:
# def take_context(question,cont):
#   ques=f'''
#   Document:{cont}
#   ___above this line is the document,below this line is what you need to do___
#   EXTRACT context related to the   question:["{question}"]. The output must be analysed from the above Document,

#   if no context related to the input exist, return:"no" only

#   question:["{question}"]

#   output:context OR "No"
#   '''

#   result = qa_chain({"query": question})


#   return result["result"]

def take_context(question,cont,model):
    context=empty_Vertex(f'''
    Document:{cont}
    ___above this line is the document,below this line is what you need to do___
    EXTRACT context related to the question. The output must be from the above Document, if no context related to the input exist, return:"no" only

    question:["{question}"]
    output:context OR "no"
    ''',model)
    return context


In [95]:
def docs_context(ques,docs):
  print("question: " + ques +"\n\nThere are "+str(len(docs))+ " sections\n")


  for i in range(len(docs)):
    res= take_context(ques,docs[i].page_content,model)
    print("\nsection "+str(i)+":\ncontext related to the question :"+ res)
    print("##################")
docs_context(questions[4],docs)

question: What are the high-level project requirements?

There are 22 sections


section 0:
context related to the question :The high-level project requirements are as follows:

- Standards and Conventions
- Project Management
- Engagement Governance
- Technical & Data Conversion Requirements
- Testing
##################

section 1:
context related to the question :This RFP document is an invitation to submit proposals for a project to replace an existing legacy software system, while providing much needed enhancements to the overall business processes. This project will be established as Project Alpha and the start date is slated for January. Timeline to implementation and cut over will be a determining factor in our evaluations.
##################

section 2:
context related to the question :no
##################

section 3:
context related to the question :We are seeking a product that will not only replace the system we have but enhance our user experience and allow us to grow and 

In [ ]:
res= take_context(questions[0],docs[0].page_content,model)
print(res)

res= take_context(questions[0],docs[1].page_content,model)
print(res)

res= take_context(questions[0],docs[2].page_content,model)
print(res)